# **Transfer Learning with ResNet50**

In [ ]:
PROJECT = "drive/MyDrive/Colab_Notebooks_Deep_Learning/100_Final_Project_DL/"

In [ ]:
import sys
sys.path.append(PROJECT)

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

import keras

from keras.models import Sequential, Model, load_model

from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from keras.utils import plot_model

from keras.models import load_model

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler

from tensorflow.keras.preprocessing import image

from tensorflow.python.ops.numpy_ops import np_config

import torch

from support_module import (load_and_preprocess_image, create_heatmap, superimpose_heatmap,predict_measurements_and_evaluate, visualize_feature_maps, TARGET_SIZE)

from tensorflow.keras.applications import ResNet50


In [ ]:
SOURCE_FEMALE = PROJECT+"/imgs_female/"
SOURCE_MALE = PROJECT+"/imgs_male/"

## **2. Tyding, cleaning and EDA**

Let's see some examples!

### **Load the datasets**

Let's load our annotations. We have 16 anthropometrics for each image.

In [ ]:
female_data = pd.read_csv(PROJECT+"/train_female.csv")
male_data = pd.read_csv(PROJECT+"/train_male.csv")

In [ ]:
female_data['id'] = female_data['id'].astype(str).str.zfill(6)
male_data['id'] = male_data['id'].astype(str).str.zfill(6)

In [ ]:
female_data['filename'] = female_data["id"].astype(str)+".png"
male_data['filename'] = male_data["id"].astype(str)+".png"

In [ ]:
# Reading and resizing images. Preprocessing for ResNet and for Custom CNN model
def preprocess_image(image_path, measurements, source):
    # Read the image
    image = tf.io.read_file(source+image_path)
    image = tf.image.decode_png(image, channels=3)

    # Resize the image to match ResNet50 input
    image = tf.image.resize(image, [257, 344])

    # Resize the image to a square by cropping the central part, then to 224x224 for Custom model
    # image = tf.image.resize_with_crop_or_pad(image, 344, 344)  # Crop to square
    # image = tf.image.resize(image, [224, 224])  # Resize to the desired input size

    # Normalize the image
    image = tf.keras.applications.resnet50.preprocess_input(image)

    return image, measurements

In [ ]:
def create_datasets(dataframe, source, batch_size, train_size, shuffle_buffer_size):
    image_paths = dataframe["filename"].values
    measurements = dataframe[["chest_circ", "waist_circ", "pelvis_circ", "neck_circ", "bicep_circ", "thigh_circ", "knee_circ",	"arm_length", "leg_length", "calf_length", "head_circ", "wrist_circ",	"arm_span", "shoulders_width", "torso_length", "inner_leg"]].values

    #Creating the tf Dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, measurements))

    # Shuffle the dataset
    dataset = dataset.shuffle(buffer_size=shuffle_buffer_size, seed=42, reshuffle_each_iteration=True)

    num_train_samples = int(len(dataset) * train_size)
    train_dataset = dataset.take(num_train_samples).map(lambda x, y: preprocess_image(x, y, source),num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    validation_dataset = dataset.skip(num_train_samples).map(lambda x, y: preprocess_image(x, y, source), num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_dataset, validation_dataset

In [ ]:
train_dataset_female, validation_dataset_female = create_datasets(female_data, SOURCE_FEMALE, batch_size=32, train_size=0.85, shuffle_buffer_size=female_data.shape[0])
train_dataset_male, validation_dataset_male = create_datasets(male_data, SOURCE_MALE, batch_size=32, train_size=0.85, shuffle_buffer_size=male_data.shape[0])

In [ ]:
base_model = tf.keras.applications.ResNet50(input_shape=(257, 344, 3), # Defining the base model and cutting classification head. We will use 'imagenet' weights.
                                            include_top=False,
                                            weights='imagenet')

base_model.trainable = False  # Freeze layers for transfer learning

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 257, 344, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 263, 350, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 129, 172, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 129, 172, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
output = tf.keras.layers.Dense(16)(x)  # No activation, direct regression

In [ ]:
transfer_model = tf.keras.Model(inputs=base_model.input, outputs=output)

### **3. Setup the training process**

We will use MLflow with Databricks to log our experiments

In [ ]:
!pip install -q mlflow databricks-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


Let's create our first experiment! Later we use the experiment ID to add runs in it. The link to the experiment is : [Initial ResNet50 with custom head](https://community.cloud.databricks.com/?o=6058138271020888#mlflow/experiments/4431290084046969?searchFilter=&orderByKey=attributes.start_time&orderByAsc=false&startTime=ALL&lifecycleFilter=Active&datasetsFilter=W10%3D&modelVersionFilter=All%20Runs&selectedColumns=attributes.%60Source%60,attributes.%60Models%60)

First we compile the model. We will use `Adam` optimizer with default hyperparamameters. Second run we tune the optimizer using modification of `Adam` - `AMSGrad` and smaller `learning_rate = 0.0005` because learning curve shows less oscillations and the the final loss is better.

In [ ]:
# Define the CosineDecay learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.0005, decay_steps=132600, alpha=0.2)


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, amsgrad=True)

In [ ]:
transfer_model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mean_absolute_percentage_error'])

We have to make some preparation before the training. In the previous runs we used `Checkpoints` if the model performance increase we will save the model.

In [ ]:
CHECKPOINT_PATH = PROJECT+"/checkpoints/male/model-{epoch:02d}-{val_loss:.2f}.keras"

We will save checkpoints for every epoch, if we have improved the loss from the previous epoch.

In [ ]:
checkpoint = ModelCheckpoint(
    CHECKPOINT_PATH,
    monitor='val_loss',   # Metric to monitor
    verbose=1,            # Verbosity level
    save_best_only=True,  # Save only the best model
    mode='min',           # `min` means the monitor metric should decrease (for 'val_loss')
    save_weights_only=False # Save the entire model
)

We will keep the logs for the training performance for later analysis with TensorBoard. We will train our model for 100 epochs, based on previous experiments, this will be more than enough.

For the last run we use `LearningRateScheduler` to decrease the learning rate after the first 10 epochs with arond 10% for every epoch till the last.

In [ ]:
# # Custom learning rate schedule
# def scheduler(epoch, lr):
#     if epoch < 10:
#         return lr
#     else:
#         return lr * tf.math.exp(-0.1)

In [ ]:
# lr_scheduler = LearningRateScheduler(scheduler)

## **4. Training**

In [ ]:
class PrintLearningRate(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        if hasattr(lr, 'numpy'):
            # If the learning rate is a TensorFlow object with a numpy method
            lr = lr.numpy()
        elif callable(lr):
            # If the learning rate is a callable (like a schedule), call it with the current step
            lr = lr(self.model.optimizer.iterations).numpy()
        print(f"Epoch {epoch+1}: Learning Rate is {lr:.6f}")


In [ ]:
transfer_model.fit(train_dataset_male,
            verbose = 1,
            validation_data=validation_dataset_male,
            callbacks=[checkpoint,
                       PrintLearningRate(),
                       TensorBoard(log_dir=PROJECT+"/checkpoints/male/logs/"),
                       ],
            # initial_epoch=1,
            epochs=100)

Epoch 1: Learning Rate is 0.000500
Epoch 1/100
1326/1326 [==============================] - ETA: 0s - loss: 2.6473 - mean_absolute_percentage_error: 3.9856
Epoch 1: val_loss improved from inf to 1.36071, saving model to drive/MyDrive/Colab_Notebooks_Deep_Learning/100_Final_Project_DL//checkpoints/male/model-01-1.36.keras
1326/1326 [==============================] - 5083s 4s/step - loss: 2.6473 - mean_absolute_percentage_error: 3.9856 - val_loss: 1.3607 - val_mean_absolute_percentage_error: 2.2609
Epoch 2: Learning Rate is 0.000500
Epoch 2/100
1326/1326 [==============================] - ETA: 0s - loss: 1.3195 - mean_absolute_percentage_error: 2.2070
Epoch 2: val_loss improved from 1.36071 to 1.31812, saving model to drive/MyDrive/Colab_Notebooks_Deep_Learning/100_Final_Project_DL//checkpoints/male/model-02-1.32.keras
1326/1326 [==============================] - 923s 696ms/step - loss: 1.3195 - mean_absolute_percentage_error: 2.2070 - val_loss: 1.3181 - val_mean_absolute_percentage_erro

In [ ]:
model = load_model("drive/MyDrive/Colab_Notebooks_Deep_Learning/100_Final_Project_DL/checkpoints/female/ResNet50/model-84-0.89.keras")

In [ ]:
model.save_weights("drive/MyDrive/Colab_Notebooks_Deep_Learning/100_Final_Project_DL/checkpoints/female/Custom/model-84-0.89.h5")